# About http 2.0
---
### 1. 개요
HTTP는 1996년 처음 1.0버전이 release되고 1999년 현재 우리가 공식적으로 가장 많이 사용하고 지원하는 버전인 1.1이 출시된 이후 15년동안 발전없이 사용되고 있다.<br>
현재의 웹은 다량의 멀티미디어 리소스를 처리 해야하고 웹페이지 하나를 구성하기 위해 다수의 비동기 요청이 발생되고 있고 이를 처리하기엔 HTTP1.1 스펙은 너무 느리고 비효율적이다. 특히 모바일 환경에선 더욱더 HTTP1.1의 스펙은 부족한감이 많다.

### 2. HTTP/1.1 동작방식
HTTP는 웹상에서 Client(IE, Chrome, Firefox 등의 브라우저)와 Server(웹서버 httpd, nginx, etc ...)간 통신을 위한 Protocol이다.<br><br>
HTTP/1.1는 기본적으로 Connection당 하나의 요청을 처리 하도록 설계 되어있다. 요청과 응답이 순차적으로 동시전송이 불가능하게 이루어 지게된다. 과정은 아래와 같다.
![Http 과정](http://www.popit.kr/wp-content/uploads/2016/11/http1-1e8d6f2a-403x600.png)


그렇다 보니 HTTP문서안에 포함된 다수의 리소스 (Images, CSS, Script)를 처리하려면 요청할 리소스 개수에 비례해서 Latency(대기 시간)는 길어지게 된다.<br><br>
위의 내용을 포함한 HTTP 1.1의 단점들은 다음과 같다.
- HOL(Head Of Line) Blocking : 특정 응답의 지연
- RTT(Round Trip Time)의 증가 : 3-way Handshake가 반복적으로 일어남
- 무거운 Header 구조(특히 cookie) 
<br><br>

결과적으로 구글은 더 빠른 Web을 실현하기 위해 throughput관점이 아닌 Latency 관점에서 HTTP를 고속화한 SPDY라 불리는 새로운 프로토콜을 구현하였다. 다만 SPDY는 HTTP를 대치하는 프로토콜이 아니고 HTTP를 통한 전송을 재 정의하는 형태로 구현이 되었다. SPDY는 실제로 HTTP/1.1에 비해 상당한 성능 향상과 효율성을 보여주었고 이는 HTTP/2 초안의 참고 규격이 된다. 

### 3. HTTP/2
위에서 설명한 것 처럼 SPDY를 기반으로 만들어진 새로운 프로토콜 구현 프로젝트이다. 이 프로젝트는 프로토콜을 완전히 다시 작성하는게 아니라 HTTP메소드, 상태 코드 및 의미는 동일하며 프로토콜을 나타 내기 위해 HTTP/1.x와 동일한 API(일부 작은 추가 기능 포함)를 사용 할 수 있어야 한다. HTTP/2의 초점은 성능에 있다. 그 중에 주요 목표 중 하나는 브라우저에서 웹 사이트로의 단일 연결을 허용하는 것이다. HTTP/2가 성능을 향상 시키는 주요 요소는 다음과 같다.

- Multiplexed <br>
한 커넥션으로 동시에 여러개의 메세지를 주고 받을 수 있으며, 응답은 순서에 상관없이 Stream으로 주고 받는다. HTTP/1.1의 Connection Keep-Alive, Pipelining의 개선이라 보면 된다.

- Stream Prioritization<br>
예를 들면 클라이언트가 요청한 HTML문서안에 CSS파일 1개와 Image파일 2개가 존재하고 이를 클라이언트가 각각 요청하고 난 후 Image파일보다 CSS파일의 수신이 늦어지는 경우 브라우저의 렌더링이 늦어지는 문제가 발생하는데 HTTP/2의 경우 리소스간 의존관계(우선순위)를 설정하여 이런 문제를 해결하고 있다.

- Server Push<br>
서버는 클라이언트의 요청에 대해 요청하지도 않은 리소스를 마음대로 보내줄 수 도 있다. 클라이언트(브라우저)가 HTML문서를 요청했고 해당 HTML에 여러개의 리소스(CSS, Image...)가 포함되어 있는경우 HTTP/1.1에서 클라이언트는 요청한 HTML문서를 수신한 후 HTML문서를 해석 하면서 필요한 리소스를 재 요청하는 반면 HTTP/2에선 Server Push기법을 통해서 클라이언트가 요청하지도 않은 (HTML문서에 포함된 리소스) 리소스를 Push 해주는 방법으로 클라이언트의 요청을 최소화 해서 성능 향상을 이끌어 낸다. 이를 PUSH_PROMISE라고 부르며 PUSH_PROMISE를 통해서 서버가 전송한 리소스에 대해선 클라이언트는 요청을 하지 않는다.

- Header Compression<br>
HTTP/2는 Header 정보를 압축하기 위해 Header Table과 Huffman Encoding 기법을 사용하여 처리하는데 이를 HPACK 압축방식이라 부르고 별도의 명세서(RFC 7531)로 관리하고 있다. 

### 4. HTTP/1.1과 HTTP/2 성능비교
두 프로토콜의 객관적인 성능비교 지표는 테스트 환경과 각각 테스트시 외부 인터넷 품질등의 영향으로 정확하게 알 수는 없지만, 일반저으로 HTTP/2를 사용만 해도 웹 응답 속도가 HTTP/1.1에 비해 15% ~ 50%가 향상 된다고 한다. <br>
 동일 개수/용량의 png이미지를 웹사이트에 로딩시켜 HTTP/1.1과 HTTP/2의 속도를 비교한 결과 효율성 차이가 90%이상 나기도 한다.

### 5. 의의
 아직까지도 대 대수의 사이트(서비스)들은 HTTP/1.1만 지원하고 있다. 하지만 우리가 모르는 사이에 많은 서비스들이 이미 HTTP/2를 적용하고 있고 잘 쓰고 있따. 아무리 서버에서 HTTP/2를 지원한다고 해도 클라이언트에서 HTTP/2를 지원하지 않는 다면 사용할 수 가 없기에 아직까지 PC용 웹페이지에서 HTTP/2사용은 미지원 클라이언트(브라우저)가 많아 한계가 있다고 생각되지만 모바일 환겨에서는 대부분의 Device가 HTTP/2를 지원하고 있고 항상 데이터 부족에 시달리는 많은 사용자들에게 보다 더 적은량으로 동일한 화면을 볼 수 있는 이득을 줄 수 있고 또한 유선망에 비해 상대적으로 느린 LTE, 3G망의 사용자에게 조금 더 나은 서비스 이용 환경을 제공 해 줄 수 있다. <br>
 물론 HTTP/2 도입을 위해 새로운 웹서버로의 변경 작업은 자칫 서비스 장애로 이어질 수 있기에 신중히 판단해서 처리할 작업이다. 